---

title: 结构方程模型多组比较分析调节变量
date: 2022-08-31 12:44:03
tags: [结构方程, mplus]


---

本教程假定你对mplus有基本的了解， 不会细致讲解每个命令， 
但是会对分组比较必要的命令进行讲解。

<!--more-->

<!-- toc -->

如果调节变量是分类变量， 或者你可以将连续变量划分为高低分组， 转换为分类变量， 
你可以使用分组比较的方法来证明调节变量的调节效应， 
比如下面是我们今天用到的模型：

<img src="imgs/multigrop-model.png">



其中， 自变量是X， 因变量是Y， M是中介变量， Z是调节变量， 调节变量是二分类变量（1代表低分组， 2代表高分组）。

### 分组比较的参数

分组比较实际上是假定每个分组的数据都有自己独立的模型， 也就是路径系数在两个分组下是独立的， 看下面的示意图， 
如果没有分组， 该模型只有三个参数， 如果使用了分组比较， 因为分组变量有两个分类， 所以总共有6个参数。

<img src="imgs/mga-parameters.png">

### 有约束和无约束模型

为了证明调节效应是成立的， 我们需要做两个模型： 

- 一个模型是有约束的， 也就是假定a1=a2, b1=b2, c1=c2
- 一个是无约束模型， 也就是假定 1 != a2 , b1 != b2, c1 != c2

如果调节效应成立，模型1的你和指数就会比模型2的差， 因为模型1限定了两组参数相等，这会违背数据所暗示的变量关系；
如果调节效应不成立， 模型1和模型2的你和指标不会差别很大， 因为数据所推理得到的两组参数差别不大， 模型增加的限制条件不会对模型拟合有很大的“伤害”。

<img src="imgs/mga-models.png">

### Mplus代码

#### 有限制模型


```
Title : MGA constrained
Data : FILE IS data.dat; !这里是你要分析数据在电脑里的路径，格式要csv或dat，首行变量名需要去掉

Variable : 
	NAMES ARE a1-a6 oc1-oc6 se1-se6 tz1-tz5 gender; !这里是数据中所有变量的名称，需要自己命名
	USEVARIABLES = a1-a6 oc1-oc6 tz1-tz5 gender; !这里是本次分析需要用到的变量名称
	GROUPING IS gender (1=man 2=woman);

ANALYSIS:
    BOOTSTRAP = 100; ! 这里根据需要设置bootstrap的抽样数，数字越大，电脑运行计算时间越长

! 下面每条路径的命名一定要加上括号，我们按照上图来命名每条路径


MODEL:
    ! 设定潜变量
    X by a1-a6(1-6);    ! 自变量
    M1 by oc1-oc6(7-12); ! 中介变量
    Y by tz1-tz5(13-17);  ! 因变量
    Y ON X (a); ! X对Y的路径我们用c'命名
    Y ON M1 (b); !M1对Y的路径命名为b1
    M1 ON X (c); !x 对M1的路径命名为a1

! 计算具体间接效应和总间接效应

MODEL CONSTRAINT:

   NEW(indrect total); !这里我们需要生成新的系数
	indrect = b * a;
	total = b * a + c;

OUTPUT:
    STAND CINT(bcbootstrap);
```

#### 无限制模型

```
Title : MGA constrained
Data : FILE IS data.dat; !这里是你要分析数据在电脑里的路径，格式要csv或dat，首行变量名需要去掉

Variable : 
	NAMES ARE a1-a6 oc1-oc6 se1-se6 tz1-tz5 gender; !这里是数据中所有变量的名称，需要自己命名
	USEVARIABLES = a1-a6 oc1-oc6 tz1-tz5 gender; !这里是本次分析需要用到的变量名称
	GROUPING IS gender (1=man 2=woman);

ANALYSIS:
    BOOTSTRAP = 100; ! 这里根据需要设置bootstrap的抽样数，数字越大，电脑运行计算时间越长



! 下面每条路径的命名一定要加上括号，我们按照上图来命名每条路径


MODEL:
    ! 设定潜变量
    X by a1-a6(1-6);    ! 自变量
    M1 by oc1-oc6(7-12); ! 中介变量
    Y by tz1-tz5(13-17);  ! 因变量


Model man: ! 对于分组为man的模型设定
    ! 设定变量关系
    Y ON X (mc); ! X对Y的路径我们用c命名
    Y ON M1 (mb); !M1对Y的路径命名为b
    M1 ON X (ma); !X对M1的路径命名为a

Model woman:! 对于分组为woman的模型设定
    Y ON X (wc); ! X对Y的路径我们用c命名
    Y ON M1 (wb); !M1对Y的路径命名为b1
    M1 ON X (wa); !X对M1的路径命名为a1

! 计算具体间接效应和总间接效应

MODEL CONSTRAINT:

	NEW( deltaa deltab deltac );
	deltaa = ma-wa;
	deltab = mb-wb;
	deltac = mc-wc;

    NEW( mindirect mtotal); !这里我们需要生成新的系数
	mindirect = ma * mb;
	mtotal = mc + ma * mb;

    NEW( windirect wtotal); !这里我们需要生成新的系数
	windirect = wa * wb;
	wtotal = wc + wa * wb;

	NEW(deltaind deltatotal );
	
	deltaind = ma * mb - wa * wb;
	deltatotal = mc + ma * mb - wc - wa * wb;


OUTPUT:
    STAND CINT(bcbootstrap);
```

### 结果解读

#### 比较卡方

- 有限制模型的卡方值：

```
Chi-Square Test of Model Fit

  Value                            517.426
  Degrees of Freedom                   263
  P-Value                           0.0000
```

- 无限制模型的卡方值：

```
Chi-Square Test of Model Fit

  Value                            483.139
  Degrees of Freedom                   254
  P-Value                           0.0000
```

- 卡方差值显著性：


我们使用这个[在线工具](https://mlln.cn/2020/09/17/%E5%8D%A1%E6%96%B9%E5%B7%AE%E5%80%BC%E7%9A%84%E6%98%BE%E8%91%97%E6%80%A7%E6%A3%80%E9%AA%8C/)来计算卡方差值的显著性， 实际上这个工具没有做什么高深的魔法，
只是将卡方和自由度计算差值， 然后查表得到显著性。

<img src="imgs/chi2diff.png">

#### 查看路径系数的差异性

知道了无限制模型显著优于有限制模型， 就证明了调节效应是存在的， 
也就是说调节变量是有意义的， 接下来我们要看到底哪个路径存在调节效应， 
在无限制模型的结果中找到这里：

```
CONFIDENCE INTERVALS OF MODEL RESULTS

                  Lower .5%  Lower 2.5%    Lower 5%    Estimate    Upper 5%  Upper 2.5%   Upper .5%
```

```
 New/Additional Parameters
    DELTAA          -0.408      -0.408      -0.408      -0.335      -0.308      -0.299      -0.259
    DELTAB          -0.057      -0.057      -0.057       0.003       0.039       0.039       0.050
    DELTAC          -0.068      -0.068      -0.068      -0.022       0.023       0.029       0.029
    MINDIREC        -0.011      -0.011      -0.011      -0.003      -0.001      -0.001       0.001
    MTOTAL          -0.098      -0.098      -0.098      -0.049      -0.019      -0.019      -0.001
    WINDIREC        -0.072      -0.072      -0.072      -0.050      -0.038      -0.037      -0.037
    WTOTAL          -0.129      -0.129      -0.121      -0.074      -0.028      -0.028      -0.028
    DELTAIND         0.035       0.035       0.035       0.047       0.062       0.063       0.063
    DELTATOT        -0.028      -0.028      -0.028       0.025       0.082       0.085       0.085
```



### 视频教程和数据下载地址

视频教程以及数据下载: https://afdian.com/item?plan_id=bbf62a802d9f11edb09552540025c377

### 咨询微信mllncn

- 参考文献[Psy 523/623 Structural Equation Modeling, Spring 2020](https://web.pdx.edu/~newsomj/semclass/ho_moderation.pdf)

- [模型图在线编辑](https://www.mlln.cn/drawio/?lightbox=1&target=blank&highlight=0000ff&edit=_blank&layers=1&nav=1&title=%E6%9C%AA%E5%91%BD%E5%90%8D%E8%A1%A8%E5%8D%95.xml#R5VrLcpswFP0als0giZeXSeq2i2amM1k0yQ5jBWgxcrEc2%2F36CiMeQpB4AkJ04o3R1fvo6F6OhIFuN8evmb%2BN7sgaJwY010cDfTYgBBZ02F9uORUWF8HCEGbxmheqDffxX8yNJrfu4zXeCQUpIQmNt6IxIGmKAyrY%2FCwjB7HYM0nEXrd%2BiCXDfeAnsvVnvKZRYfWgW9u%2F4TiMyp6BsyhyVn7wO8zIPuX9GRA9n39F9sYv2%2BIT3UX%2BmhwaJrQ00G1GCC2eNsdbnOTYlrAV9b705FbjznBKL6nAJ7Sjp3LqeM2Q4MmUpOzv5jwfnNcwWSqim4Q9AvbIOslOD9x%2BTjzmiSubJX9hSk98Zf09JcxEMhqRkKR%2B8p2QbdnGMaYPVa089chziqHl4%2BmdHDftyD4LeClONOpnIealrApYRlhMNpgNlBXJcOLT%2BEVs3efMCatyNXrsgQPYDSbv%2BsVP9rzROwneGst8iocopvh%2B658Hf2B7ScSXt4czio%2BvYyDPjldAJdNOJVV5%2BlDzGpjcFjU4bZvDAXFemX4vlQoelGQC76ASaBCpaO59VEIyleBEVPKG7cteIC6FcgBgA9DhVX%2BQmHVSUdi2RQojp0XNYoF4rRbG1TAugh1JO%2FhB%2Bw5etGaP5A286Ni%2FbZDew0JLguNROxy29TYelZMbG5DFKNvSvHKbsc6cZl%2BKjmzoVr0UMQAGQaYWF2AORKHbYUFPpKgUS8dzWAB%2BPHgtezp40ceDF7UDjrpwW7rpRoB50h5gkIXYq5JAMFcOMW5HhLFGiDBAjrmrYS5UrUh7JinlDVhmP5UF3tpyOAKLi5dp%2FBBlS5DPQLchcRfaU%2Bo24EiIGNBJKF9vARnnz56UGZ92ZyZcswKWuT3WmewpzP99ULbDhlA0VWS8gvcchKJAX6%2BDvrZG%2BrrSYgUDPcZg%2Bfger%2BCND2GPnnTEreUqDHCetDb6BWXlNqrD2ekUZenn5yUp7bcBUSYpq5PguWnKN32egg17MWhaVaVw3GxN5bbaB0GOOtkDtapKPfBaznTwalWVmuBtBx11Qbfj2PLpDMmgONN4pWEdjPOOb7eVpjOh0oSy7FkN2%2FezU5rlBZRw8GlqDFuyrpqB0oRei4UenFBrQlm%2BjKQ14f%2BvNcsLB%2BEK0tFIYFnPBAN9hg6tCRWcNvXdXbY3FzBddZFPVlf65eZCRsCaTnCW7n5egtO6CBNlmhONcz6kVnN2%2Bj6Nx8TlJ2z639qRgtPGvpvMNlEXCj%2B%2BmI0smg5gS4oPKgG2Ph7AqCP8qAvASNZUufSUPYd%2B6QlF1nnTCU%2FUcbu0dIxrz7gGxtI2FkvDu6kuilZZ4%2FKIEQ%2FKGas8Y9WREeQZQUfGaOvReOMcdibfOtwDsOu%2BD3asCDTHWJIOESYtSQeO%2BX2e2bMk0OxZEmj%2BF0uCXMlzKF0Ulqy%2F%2By58T%2F1xPVr%2BAw%3D%3D)